In [1]:
import numpy as np
import os, sys
import importlib
from tqdm.notebook import tqdm, trange
import pandas as pd
import json
import itertools
from IPython.display import display

from FewShotTestHandler import FewShotTestHandler, extract_test_result_sequence
from dataset import DatasetHandler
from classifier import WeightedTextFewShotClassifier
from video_clip.video_clip import VideoClipVLM

/home/rob/vlm_benchmark/FewShotTestHandler.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
VIDEOCLIP_NUM_SECONDS_LIST = [2, 4, 6]
VIDEOCLIP_SAMPLE_STRAT_LIST = ["center", "start", "spread"]

DATASET_NAME_LIST = ["moma_act", "moma_sact", "kinetics_100"]
DATASET_SPLIT_LIST = ["test"]
N_WAY_LIST = [5, 10, 20]
N_SUPPORT_LIST = [0, 1, 5, 10, 20]
N_QUERY = 1
N_EPISODES = 1000

CLASSIFIER_TEXT_WEIGHT_LIST = [0, 0.1, 0.5, 1.0, 2.0, 4.0, 10.0]

In [3]:
param_iter = list(itertools.product(
    VIDEOCLIP_NUM_SECONDS_LIST,
    VIDEOCLIP_SAMPLE_STRAT_LIST,
    DATASET_NAME_LIST,
    DATASET_SPLIT_LIST,
    N_WAY_LIST,
    N_SUPPORT_LIST,
    CLASSIFIER_TEXT_WEIGHT_LIST
))

# Filter out param elements with 0-shot non-1 text weight
param_iter = list(filter(lambda x: x[5] != 0 or x[6] == 1, param_iter))

In [4]:
test_handler = FewShotTestHandler()

vlm = None
dataset = None
cached_dataset_ids = set()
pbar = tqdm(param_iter)
for params in pbar:
    num_seconds, sample_strat, dataset_name, dataset_split, n_way, n_support, text_weight = params
    pbar.set_postfix(dict(num_seconds=num_seconds, sample_strat=sample_strat, dataset_name=dataset_name, dataset_split=dataset_split,
                          n_way=n_way, n_support=n_support, text_weight=text_weight))
    
    if vlm is None or not (vlm.num_seconds == num_seconds and vlm.sample_strat == sample_strat):
        vlm = VideoClipVLM(num_seconds=num_seconds, sample_strat=sample_strat)
        cached_dataset_ids = set()
        
    if dataset is None or not (dataset.name == dataset_name and dataset.split == dataset_split):
        dataset = DatasetHandler(name=dataset_name, split=dataset_split)
        
    if dataset.id() not in cached_dataset_ids:
        test_handler.fill_cache(vlm, dataset)
        cached_dataset_ids.add(dataset.id())
        
    if not dataset.valid_for_few_shot(n_way, n_support, N_QUERY):
        continue
    
    classifier = WeightedTextFewShotClassifier(vlm, text_weight=text_weight)
    test_handler.run_few_shot_test(classifier, dataset, n_way, n_support, N_QUERY, N_EPISODES)

  0%|          | 0/2349 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MMBertForEncoder: ['bert.encoder.layer.9.attention.self.query.bias', 'bert.encoder.layer.6.attention.self.key.weight', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encoder.layer.6.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.self.value.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.key.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.6.output.dense.bias', 'bert.encoder.layer.8.attention.self.value.bias', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.9.attention.self.value.bias', 'bert.encoder.layer.9.attention.output.LayerNorm.weight', 'bert.encoder.layer.8.intermediate.dense.bias', 'bert.encoder.layer.9.attention.

dataset.moma.momaapi.lookup._read_anns() took 1.0858867168426514 sec
dataset.moma.momaapi.statistics._read_statistics() took 0.0006718635559082031 sec


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]